In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.signal
from scipy.signal import welch, upfirdn, firwin, firwin2, lfilter
import scipy.special
from scipy.special import erfc
import math
import scipy.signal as signal
from math import log, log2, sqrt
import ipywidgets as widgets
from ipywidgets import IntSlider, IntRangeSlider, FloatSlider, interactive, Layout, Dropdown, IntText, HBox, VBox, Output
from IPython.display import display, clear_output
from commpy.channels import awgn
import time
import warnings
warnings.filterwarnings('ignore')
print("Libraries added successfully!")

Libraries added successfully!


In [2]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html1 = widgets.HTML(
  value=loading
)
timer_html1 = widgets.HTML(
    value="Elapsed time: - seconds"
)

# Parameters
bps = 4  # bits per symbol
Nsymb = 1000  # number of simulated symbols
ns = 80  # number of samples per symbol (oversampling)

# Derived parameters
M = 2 ** bps  # number of different symbols
BR = 1  # Baud Rate
fc = 2 * M * BR  # RF frequency

# Derived parameters
nb = bps * Nsymb  # number of simulated data bits
T = 1 / BR  # one symbol period
Ts = T / ns  # oversampling period

# M frequencies in "coherent" distance (BR)
f = fc + (BR/2) * (np.arange(1, M + 1) - (M + 1) / 2)

# Calculate the maximum frequency
fmax = np.max(f)

# Recalculate ns to ensure Fs = ns * BR > 2 * fmax
Fs = 2 * fmax
ns = int(np.ceil(Fs / BR)) + 10

# Recalculate the oversampling period
Ts = T / ns

# input data bits
y = np.random.randint(0, 2, nb)
x = y.reshape((Nsymb, bps))

t = np.arange(0, len(x) * T, T)  # time vector on the T grid
tks = np.arange(0, T, Ts)  # oversampling time vector

# FSK signal
s = []
A = np.sqrt(2 / T / ns)
for k in range(len(x)):
    fk = f[int(''.join(map(str, x[k])), 2)]
    tk = (k * T) + tks
    s.append(np.sin(2 * np.pi * fk * tk))
s = np.concatenate(s)


def calculate_errors(EbNo_range):
    # Start timer
    loader_html1.value = loading
    start_time = time.time()

    clear_output(wait=True)
    EbNo_values = list(range(EbNo_range[0], EbNo_range[1] + 1))
    errors_list = []
    for EbNo in EbNo_values:
        # Calculate SNR
        SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

        # add noise to the FSK (passband) signal
        noisy_signal = awgn(s, SNR)

        # FSK receiver
        xr = []
        for k in range(len(noisy_signal) // ns):
            tk = (k * T) + tks
            sk = noisy_signal[k * ns:(k + 1) * ns]
            smi = []
            for fi in f:
                si = np.sin(2 * np.pi * fi * tk)
                smi.append(np.sum(sk * si))
            j = np.argmax(smi)
            xr.append([int(bit) for bit in bin(j)[2:].zfill(bps)])
        xr = np.array(xr).reshape((Nsymb, bps))

        # count errors
        errors = np.sum(x != xr)
        errors_list.append(errors)

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(EbNo_values, errors_list, marker='o', linestyle='-', markersize=8)
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('Number of Errors')
    plt.title('Number of Errors vs. Eb/No')
    plt.grid(True)
    plt.show()
    
    # Show elapsed time
    elapsed_time = time.time() - start_time
    timer_html1.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html1.value = done

# Create the range slider widget
EbNo_slider = IntRangeSlider(
    value=[0, 20],
    min=0,
    max=20,
    step=1,
    description='EbNo (dB):',
    continuous_update=False,
    layout=Layout(width='99%')
)

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html1, timer_html1], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox for the input widgets (similar to the first code snippet)
input_widgets = widgets.VBox([EbNo_slider], layout=widgets.Layout(flex='1 1 auto', width='auto'))

# Create an HBox to combine inputs and loader timer box, with the same layout style
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Create the interactive output for the calculate_errors function
out = widgets.interactive_output(calculate_errors, {'EbNo_range': EbNo_slider})

# Display the UI and output
clear_output(wait=True)  # Clear the previous output
display(ui, out)

Output()

In [3]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html2 = widgets.HTML(
  value=loading
)
timer_html2 = widgets.HTML(
    value="Elapsed time: - seconds"
)


def fsk_errors(bps, Nsymb, ns, EbNo):
    # Input parameters
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    fc = 2 * M * BR  # RF frequency

    # Derived parameters
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period

    # M frequencies in "coherent" distance (BR)
    f = fc + (BR/2) * (np.arange(1, M + 1) - (M + 1) / 2)

    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # awgn channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)
    x = y.reshape((Nsymb, bps))

    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)  # oversampling time vector

    # FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        fk = f[int(''.join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))
    s = np.concatenate(s)

    # add noise to the FSK (passband) signal
    s = awgn(s, SNR)

    # FSK receiver
    xr = []
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        smi = []
        for fi in f:
            si = np.sin(2 * np.pi * fi * tk)
            smi.append(np.sum(sk * si))
        j = np.argmax(smi)
        xr.append([int(bit) for bit in bin(j)[2:].zfill(bps)])
    xr = np.array(xr).reshape((Nsymb, bps))

    # count errors
    errors = np.sum(x != xr)
    return errors


def fsk_errors_non_coh(bps, Nsymb, ns, EbNo):
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    fc = 2 * M * BR  # RF frequency

    # Derived parameters
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period
    f = fc + BR * (np.arange(1, M + 1) - (M + 1) / 2)  # M frequencies
    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # AWGN channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)  # Ensure integer dimensions
    x = y.reshape((-1, bps))
    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)

    # FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        fk = f[int("".join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s = np.concatenate((s, np.sin(2 * np.pi * fk * tk)))

    # add noise to the FSK (passband) signal
    s = awgn(s, SNR)

    # FSK receiver
    # Non-coherent demodulation
    xr = []
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        sm = []
        for i in range(M):
            si = np.sin(2 * np.pi * (f[i] * tk))
            sq = np.cos(2 * np.pi * (f[i] * tk))
            smi = np.sum(sk * si[:len(sk)])
            smq = np.sum(sk * sq[:len(sk)])
            sm.append(np.sqrt(smi ** 2 + smq ** 2))
        j = np.argmax(sm)
        xr = np.concatenate((xr, np.array(list(np.binary_repr(j, width=bps)), dtype=int)))
    # count errors
    x_reshaped = x.reshape(-1)
    xr_reshaped = xr.reshape(-1)
    err = np.not_equal(x_reshaped, xr_reshaped[:len(x_reshaped)])
    errors = np.sum(err)
    return errors


# BER simulation function
def simulate_ber(bps, Nsymb, ns, EbNo_values, coherent=True):
    ber = []
    for EbNo in EbNo_values:
        errors = 0
        for _ in range(1):  # averaging over 10 iterations
            if coherent:
                errors += fsk_errors(bps, Nsymb, ns, EbNo)
            else:
                errors += fsk_errors_non_coh(bps, Nsymb, ns, EbNo)
        ber.append(errors / (Nsymb * bps))
    return ber


# Define the BER values directly inside the functions
def theoretical_ber_coh(EbNo, M):
    if M == 2:
        return np.array([0.15866, 0.13093, 0.10403, 0.078896, 0.056495,
    0.037679, 0.023007, 0.012587, 0.0060044, 0.0024133,
    0.0007827, 0.00019399, 3.4303e-05, 3.9692e-06, 2.6951e-07])
    elif M == 4:
        return np.array([0.11814, 0.087789, 0.060786, 0.038512, 0.021824, 
    0.010751, 0.0044428, 0.0014733, 0.00037102, 6.6229e-05, 
    7.6892e-06, 5.2118e-07, 1.7997e-08, 2.6653e-10, 1.362e-12])
    elif M == 8:
        return np.array([0.10227, 0.067834, 0.041318, 0.022024, 0.0099156, 
    0.0036087, 0.0010058, 0.00020086, 2.6486e-05, 2.0836e-06, 
    8.6119e-08, 1.5924e-09, 1.074e-11, 2.0391e-14, 7.8524e-18])
    elif M == 16:
        return np.array([0.089859, 0.055663, 0.029957, 0.013469, 0.004819,
    0.001293, 0.00024205, 2.897e-05, 1.9921e-06, 6.8928e-08,
    1.0145e-09, 5.1257e-12, 6.7629e-15, 1.6453e-18, 4.6157e-23])
    elif M == 32:
        return np.array([0.082719, 0.047105, 0.022469, 0.0085348, 0.0024266,
    0.00047917, 6.0083e-05, 4.2975e-06, 1.5388e-07, 2.3425e-09,
    1.2294e-11, 1.6992e-14, 4.3826e-18, 1.3266e-22, 2.1688e-28])
    else:
        raise ValueError("Unsupported value of M for coherent case")


def theoretical_ber_non_coh(EbNo, M):
    if M == 2:
        return np.array([0.30327, 0.26644, 0.22637, 0.18438, 0.1424, 0.10287, 
    0.068311, 0.0408, 0.021324, 0.0094212, 0.003369, 
    0.0009231, 0.00018089, 2.3244e-05, 1.7558e-06])
    elif M == 4:
        return np.array([0.22934, 0.18475, 0.13987, 0.097719, 0.061557, 
    0.033946, 0.01579, 0.0059139, 0.0016837, 0.00033939, 
    4.4371e-05, 3.3753e-06, 1.3045e-07, 2.1593e-09, 1.233e-11])
    elif M == 8:
        return np.array([0.19472, 0.14559, 0.099187, 0.059806, 0.030757, 
    0.012878, 0.0041438, 0.00095467, 0.00014449, 1.2945e-05, 
    6.0428e-07, 1.2541e-08, 9.4638e-11, 2.0092e-13, 8.6607e-17])
    elif M == 16:
        return np.array([0.17469, 0.12169, 0.074737, 0.038861, 0.01625, 
    0.005127, 0.0011288, 0.00015786, 1.2538e-05, 4.943e-07, 
    8.2001e-09, 4.6432e-11, 6.8517e-14, 1.8682e-17, 6.0826e-22])
    elif M == 32:
        return np.array([0.16103, 0.10471, 0.058014, 0.025984, 0.0088058, 
    0.0020817, 0.00031127, 2.6219e-05, 1.0859e-06, 1.8789e-08, 
    1.1086e-10, 1.7154e-13, 4.9582e-17, 1.737e-21, 4.2722e-27])
    else:
        raise ValueError("Unsupported value of M for non-coherent case")


# Function to update the plot
def update_plot(bps):
    # Start the timer
    loader_html2.value = loading
    start_time = time.time()

    Nsymb = 2000  # number of symbols
    ns = 100  # oversampling factor
    EbNo_dB_sim = np.arange(0, 10, 2)  # Eb/No values in dB for simulation
    EbNo_dB_theory = np.arange(0, 15, 1)  # Eb/No values in dB for theoretical
    
    EbNo_sim = 10 ** (EbNo_dB_sim / 10)  # convert dB to linear for simulation
    EbNo_theory = 10 ** (EbNo_dB_theory / 10)  # convert dB to linear for theoretical
    M = 2 ** bps  # M-ary FSK

    # Simulate BER
    ber_coh_sim = simulate_ber(bps, Nsymb, ns, EbNo_dB_sim, coherent=True)
    ber_non_coh_sim = simulate_ber(bps, Nsymb, ns, EbNo_dB_sim, coherent=False)
    
    # Theoretical BER
    ber_coh_theory = theoretical_ber_coh(EbNo_theory, M)
    ber_non_coh_theory = theoretical_ber_non_coh(EbNo_theory, M)

    # Show plot
    plt.figure(figsize=(10, 6))
    plt.semilogy(EbNo_dB_sim, ber_coh_sim, 'o', label=f'Simulated Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_sim, ber_non_coh_sim, 's', label=f'Simulated Non-Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_theory, ber_coh_theory, label=f'Theoretical Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_theory, ber_non_coh_theory, label=f'Theoretical Non-Coherent {M}-FSK')

    plt.title(f'{M}-FSK System')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.grid(True, which='both')
    plt.legend()
    plt.show()

    # Show elapsed time
    elapsed_time = time.time() - start_time
    timer_html2.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html2.value = done

# Dropdown for M-FSK selection
bps_dropdown = Dropdown(
    options=[('2-FSK', 1), ('4-FSK', 2), ('8-FSK', 3), ('16-FSK', 4), ('32-FSK', 5)],
    value=2,
    description='M-FSK:',
    style={'description_width': 'initial'}, 
    continuous_update=False
)

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html2, timer_html2], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox for the input widgets (similar to the first code snippet)
input_widgets = widgets.VBox([bps_dropdown], layout=widgets.Layout(flex='1 1 auto', width='auto'))

# Create an HBox to combine inputs and loader timer box, with the same layout style
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Create the interactive output for the update_plot function
out = widgets.interactive_output(update_plot, {'bps': bps_dropdown})

# Create a VBox to hold the UI and the plot output, with a centered layout
ui_container = widgets.VBox([ui, out], layout=widgets.Layout(display='flex', justify_content='center', width='100%', align_items='center'))

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui_container)


In [4]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)


# Define the function to be called when widgets change
def update_plot(fsklvl, Nsymb, EbNo):
    # Start the timer
    loader_html3.value = loading
    start_time = time.time()

    clear_output(wait=True)
    
    bps = int(math.log2(fsklvl))
    # Derived parameters for coherent FSK
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    T = 1 / BR  # one symbol period
    fc = 2 * M * BR  # RF frequency
    ns = 80  # samples per symbol
    nb = bps * Nsymb  # number of simulated data bits
    f = fc + (BR / 2) * ((np.arange(1, M + 1)) - (M + 1) / 2)
    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # awgn channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)
    x = np.reshape(y, (len(y) // bps, bps))

    # time vectors
    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)  # oversampling time vector

    # FSK signal generation
    s = []
    A = np.sqrt(2 / T / ns)

    for k in range(len(x)):
        fk = f[int(''.join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))

    s_coherent = np.concatenate(s)

    # Plotting power spectral density for coherent FSK
    frequencies_coherent, Pxx_coherent = welch(s_coherent, fs=ns*BR, nperseg=50000, noverlap=25000)

    # Derived parameters for non-coherent FSK
    ns = 90  # samples per symbol
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period

    # Frequencies in "non-coherent" distance
    f = fc + BR * ((np.arange(1, M + 1)) - (M + 1) / 2)
    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # awgn channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)
    x = np.reshape(y, (len(y) // bps, bps))

    # time vectors
    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)  # oversampling time vector

    # FSK signal generation
    s = []
    A = np.sqrt(2 / T / ns)

    for k in range(len(x)):
        fk = f[int(''.join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))

    s_non_coherent = np.concatenate(s)

    # Plotting power spectral density for non-coherent FSK
    frequencies_non_coherent, Pxx_non_coherent = welch(s_non_coherent, fs=ns*BR, nperseg=50000)

    # Plot both side by side
    fig, axs = plt.subplots(2, 1, figsize=(10, 8))

    axs[0].semilogy(frequencies_coherent, Pxx_coherent, linewidth=0.5)
    axs[0].set_title("FSK Coherent")
    axs[0].set_xlabel("Frequency (Hz)")
    axs[0].set_ylabel("Power Spectral Density (dB/Hz)")
    axs[0].grid(True)

    axs[1].semilogy(frequencies_non_coherent, Pxx_non_coherent, linewidth=0.5)
    axs[1].set_title("Non Coherent FSK")
    axs[1].set_xlabel("Frequency (Hz)")
    axs[1].set_ylabel("Power Spectral Density (dB/Hz)")
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()
    
    # Stop the timer and update the timer HTML
    elapsed_time = time.time() - start_time
    timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    # Update the loading animation to done
    loader_html3.value = done

fsklvl_dropdown=Dropdown(options=[2, 4, 8, 16, 32], value=4, description='FSK levels', style={'description_width': 'initial'}, layout=Layout(flex='1 1 auto', width='auto'), continuous_update=False)
Nsymb_slider=IntSlider(value=10000, min=1000, max=50000, step=1000, description='Nsymb', style={'description_width': 'initial'}, layout=Layout(flex='1 1 auto', width='auto'), continuous_update=False)
EbNo_slider=IntSlider(value=8, min=0, max=20, step=1, description='EbNo', style={'description_width': 'initial'}, layout=Layout(flex='1 1 auto', width='auto'), continuous_update=False)

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox for the input widgets (similar to the first code snippet)
input_widgets = widgets.VBox([fsklvl_dropdown, Nsymb_slider, EbNo_slider], layout=widgets.Layout(flex='1 1 auto', width='auto'))

# Create an HBox to combine inputs and loader timer box, with the same layout style
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Create the interactive output for the update_plot function
out = widgets.interactive_output(update_plot, {'fsklvl': fsklvl_dropdown, 'Nsymb': Nsymb_slider, 'EbNo': EbNo_slider})

# Display the UI and output
clear_output(wait=True)  # Clear the previous output
display(ui, out)

Output()

In [5]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html4 = widgets.HTML(
  value=loading
)
timer_html4 = widgets.HTML(
    value="Elapsed time: - seconds"
)


# Define AWGN function to simulate noise
def awgn(signal, SNR):
    snr_linear = 10 ** (SNR / 10)
    signal_power = np.mean(signal ** 2)
    noise_power = signal_power / snr_linear
    noise = np.sqrt(noise_power) * np.random.normal(size=signal.shape)
    return signal + noise

# MSK error function
def msk_errors(Nbits, nsamp, EbNo):
    n = Nbits  # number of data bits
    R = 2000000  # bit rate
    fc = 8000000  # carrier frequency
    ns = nsamp  # oversampling factor

    # AWGN channel
    SNR = EbNo - 10 * np.log10(ns/2)  # in dB
    T = 1 / R  # 1-bit period (= basic period)
    Ts = T / ns  # sampling frequency
    fss = 1/Ts

    # Input sequence
    y = np.concatenate(([1], np.sign(np.random.rand(n - 1) - 0.5)))  # random numbers, -1 or 1
    x = y

    g = np.ones(ns)
    xx = upfirdn(g, x, up=ns)  # NRZ polar pulse train samples

    # Time grid
    ts = np.arange(0, len(xx) * Ts, Ts)  # of length ns*(n+1)

    ## MSK TRANSMITTER
    xs = xx
    theta = np.cumsum(xs) * np.pi / 2 / ns
    xs_i = np.cos(theta)  # in-phase component
    xs_q = np.sin(theta)  # quadrature component

    # Ensure that xs_i and xs_q are the same length as the time grid `ts`
    if len(xs_i) > len(ts):
        xs_i = xs_i[:len(ts)]
        xs_q = xs_q[:len(ts)]
    elif len(ts) > len(xs_i):
        ts = ts[:len(xs_i)]

    # Modulation
    s = xs_i * np.cos(2 * np.pi * fc * ts) - xs_q * np.sin(2 * np.pi * fc * ts)

    # Addition of noise
    s = awgn(s, SNR)

    ## MSK RECEIVER
    xs_i = s * np.cos(2 * np.pi * fc * ts)
    xs_q = -s * np.sin(2 * np.pi * fc * ts)

    # LP (Parks-McClellan) filter
    f1 = 0.75*(fss/2)/ns
    f2 = 4*f1
    order = 8 * ns
    fpts = [0, f1, f2, fss/2]
    mag = [1, 1, 0, 0]
    wt = [1, 1]
    b = firwin2(order+1, fpts, mag, fs=fss)
    a = 1

    len_xs_i = len(xs_i)
    dummy = np.concatenate((xs_i, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    delay = order // 2
    xs_i = dummy1[delay:delay + len_xs_i]
    xs_i = np.concatenate((xs_i, np.ones(nsamp-1)))

    dummy = np.concatenate((xs_q, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    xs_q = dummy1[delay:delay + len_xs_i]

    bi = 1
    xr_1 = 1
    xr = np.zeros(n)
    for k in range(0, n, 2):
        li = np.arange((k+1) * ns, min((k + 3) * ns-1, len(xs_i)))
        lq = np.arange(k * ns, min((k + 2) * ns-1, len(xs_q)))
        xi = xs_i[li]
        xq = xs_q[lq]
        gmi = np.cos(np.pi / 2 / T * Ts * li)  # matched-filter pulse
        gmq = -gmi  # =sin(pi/2/T*Ts*lq);
        bi_1 = bi
        bi = np.sign(np.sum(xi * gmi))
        bq = np.sign(np.sum(xq * gmq))
        xr[k] = bi_1 * bq
        xr[k+1] = bi * bq
        xr_1 = xr[k + 1]

    xr = xr.reshape(-1)
    err = np.not_equal(x, xr)
    errors = np.sum(err)
    return errors / Nbits

# MSK error function with precoding
def msk_errors_precoding(Nbits, nsamp, EbNo):
    n = Nbits  # number of data bits
    R = 2000000  # bit rate
    fc = 8000000  # carrier frequency
    ns = nsamp  # oversampling factor

    # AWGN channel
    SNR = EbNo - 10 * np.log10(ns/2)  # in dB
    T = 1 / R  # 1-bit period (= basic period)
    Ts = T / ns  # sampling frequency
    fss = 1/Ts

    # Input sequence
    y = np.concatenate(([1], np.sign(np.random.rand(n - 1) - 0.5)))  # random numbers, -1 or 1
    x = y
    x[0] = 1
    for i in range(1, len(y)):
        x[i] = y[i] * x[i-1]  # Apply precoding rule

        

    g = np.ones(ns)
    xx = upfirdn(g, x, up=ns)  # NRZ polar pulse train samples

    # Time grid
    ts = np.arange(0, len(xx) * Ts, Ts)  # of length ns*(n+1)

    ## MSK TRANSMITTER
    xs = xx
    theta = np.cumsum(xs) * np.pi / 2 / ns
    xs_i = np.cos(theta)  # in-phase component
    xs_q = np.sin(theta)  # quadrature component

    # Ensure that xs_i and xs_q are the same length as the time grid ts
    if len(xs_i) > len(ts):
        xs_i = xs_i[:len(ts)]
        xs_q = xs_q[:len(ts)]
    elif len(ts) > len(xs_i):
        ts = ts[:len(xs_i)]

    # Modulation
    s = xs_i * np.cos(2 * np.pi * fc * ts) - xs_q * np.sin(2 * np.pi * fc * ts)

    # Addition of noise
    s = awgn(s, SNR)

    ## MSK RECEIVER
    xs_i = s * np.cos(2 * np.pi * fc * ts)
    xs_q = -s * np.sin(2 * np.pi * fc * ts)

    # LP (Parks-McClellan) filter
    f1 = 0.75*(fss/2)/ns
    f2 = 4*f1
    order = 8 * ns
    fpts = [0, f1, f2, fss/2]
    mag = [1, 1, 0, 0]
    wt = [1, 1]
    b = firwin2(order+1, fpts, mag, fs=fss)
    a = 1

    len_xs_i = len(xs_i)
    dummy = np.concatenate((xs_i, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    delay = order // 2
    xs_i = dummy1[delay:delay + len_xs_i]
    xs_i = np.concatenate((xs_i, np.ones(nsamp-1)))

    dummy = np.concatenate((xs_q, np.zeros(order)))
    dummy1 = lfilter(b, a, dummy)
    xs_q = dummy1[delay:delay + len_xs_i]

    # Updated MSK decoding for precoded bits with recursive logic
    bi = 1
    xr_1 = 1  # Initialize previous decoded bit (xr_1)
    xr = np.zeros(n)  # Array to store decoded bits

    for k in range(0, n, 2):
        li = np.arange((k+1) * ns, min((k + 3) * ns-1, len(xs_i)))
        lq = np.arange(k * ns, min((k + 2) * ns-1, len(xs_q)))
        xi = xs_i[li]
        xq = xs_q[lq]
        
        # Matched filter output (to match MSK modulation characteristics)
        gmi = np.cos(np.pi / 2 / T * Ts * li)  # In-phase matched filter pulse
        gmq = -gmi  # Quadrature matched-filter pulse (sin is negative of cosine)
        
        # Save previous in-phase matched filter output
        bi_1 = bi
        
        # Decode in-phase (I) and quadrature (Q) components
        bi = np.sign(np.sum(xi * gmi))
        bq = np.sign(np.sum(xq * gmq))
        
        # Apply recursive decoding rule for precoded MSK
        xr[k] = bi_1 * bq  # Decode the k-th bit
        xr[k+1] = bi * bq  # Decode the (k+1)-th bit
        
        # Update the previously decoded bit (xr_1)
        xr_1 = xr[k+1]

    xr = xr.reshape(-1)
    err = np.not_equal(y, xr)
    errors = np.sum(err)
    return 0.5*errors / Nbits

# Function to simulate BER and plot
def update_msk_plot(nsamp):
    if loader_html4.value != loading:
        loader_html4.value = loading  # Set loading indicator only if not already set
    start_time = time.time()

    EbNo_range = np.arange(0, 9, 1)  # EbNo from 0 to 10 dB
    Nbits = 10000  # Increase number of bits to reduce variance

    simulated_BER = []
    simulated_BER_precoding = []
    theoretical_BER = []
    theoretical_BER_precoded = []
    
    for EbNo in EbNo_range:
        # Simulate both with and without precoding
        sim_BER = msk_errors(Nbits, nsamp, EbNo)
        sim_BER_precoded = msk_errors_precoding(Nbits, nsamp, EbNo)
        
        # Theoretical BER without precoding
        theoretical_BER_value = 0.9 * erfc(np.sqrt(10**(EbNo / 10)))
        # Theoretical BER with precoding
        theoretical_BER_precoded_value = erfc(np.sqrt(10**(EbNo / 10))) / 2
        
        simulated_BER.append(sim_BER)
        simulated_BER_precoding.append(sim_BER_precoded)
        theoretical_BER.append(theoretical_BER_value)
        theoretical_BER_precoded.append(theoretical_BER_precoded_value)

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.semilogy(EbNo_range, simulated_BER, 'o', label='Simulated BER (without precoding)')
    plt.semilogy(EbNo_range, simulated_BER_precoding, 's', label='Simulated BER (with precoding)')
    plt.semilogy(EbNo_range, theoretical_BER, label='Theoretical BER (without precoding)')
    plt.semilogy(EbNo_range, theoretical_BER_precoded, label='Theoretical BER (with precoding)')
    plt.xlabel('$E_b/N_0$ (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.title('MSK Modulation (with and without precoding)')
    plt.legend()
    plt.grid(True, which='both')
    plt.show()

    # Update timer and loading indicator
    elapsed_time = time.time() - start_time
    timer_html4.value = f"Elapsed time: {elapsed_time:.2f} seconds"
    loader_html4.value = done


# Interactive UI components
nsamp_slider = widgets.IntSlider(
    value=32,
    min=16,
    max=128,
    step=16,
    description='Oversampling Factor:',
    layout=Layout(width='auto', flex='1 1 auto'),
    style={'description_width': 'initial'}, 
    continuous_update=False
)

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html4, timer_html4], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

# Create a VBox for the input widgets (similar to the first refactored code)
input_widgets = widgets.VBox([nsamp_slider], layout=widgets.Layout(flex='1 1 auto', width='auto'))

# Create an HBox to combine inputs and loader timer box, with the same layout style
ui = widgets.HBox([input_widgets, loader_timer_box], layout=widgets.Layout(align_items='center'))

# Create the interactive output for the update_msk_plot function
out = widgets.interactive_output(update_msk_plot, {'nsamp': nsamp_slider})

# Display the UI and output
clear_output(wait=True)  # Clear the previous output
display(ui, out)

Output()